# VGG-16 training for 21 Activities

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

Defining common Keras functions

In [3]:
from __future__ import division
from keras.applications import vgg16
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from easydict import EasyDict as edict
import numpy as np
from keras.constraints import maxnorm

import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.8):
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session())

img_width, img_height = 224, 224

def define_model(img_width = 224, img_height = 224):
    base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))    
    # The only trainable layers are the Fully Convolutional
    for layer in base_model.layers:
        layer.trainable = False
        
    x = base_model.output
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(21, activation='softmax', name='predictions')(x)
        
    return Model(input=base_model.input, output=x)

def define_dropout_model(img_width = 224, img_height = 224):
    base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
    for layer in base_model.layers:
        layer.trainable = True
        
    x = base_model.output        
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1', W_constraint=maxnorm(3))(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2', W_constraint=maxnorm(3))(x)
    x = Dropout(0.5)(x)
    x = Dense(21, activation='softmax', name='predictions')(x)
        
    return Model(input=base_model.input, output=x)

class HistoryLog(Callback):
    def on_train_begin(self,logs={}):
        self.training = edict({'loss': []})
        self.epoch = edict({'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []})

    def on_epoch_end(self, epoch, logs={}):        
        self.epoch.acc.append(logs.get('acc'))
        self.epoch.loss.append(logs.get('loss'))
        self.epoch.val_acc.append(logs.get('val_acc'))
        self.epoch.val_loss.append(logs.get('val_loss'))
        
    def on_batch_end(self, batch, logs={}):
        self.training.loss.append(logs.get('loss'))
        
    def log_training_loss(self, filepath):
        training_loss = np.array(self.training.loss)
        np.savetxt(filepath, training_loss, delimiter=",")
        
    def log_epoch(self, filepath):
        epoch = np.asarray([self.epoch.loss, self.epoch.val_loss, self.epoch.acc, self.epoch.val_acc])
        np.savetxt(filepath, epoch.T, delimiter=",")

Using TensorFlow backend.


Printing VGG-16 adjusted architecture

In [4]:
model = define_dropout_model()
model.summary()
K.clear_session()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [14]:
def makedirs(dir_path):
    try:
        os.makedirs(dir_path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(dir_path):
            pass
        else:
            raise
            
def num_digits(number):
    return int(np.floor(np.log10(np.abs(number))) + 1)

def ext(path):
    return os.path.splitext(path)[-1]

In [14]:
def link_images(num_categories, split_dir, padding_zeros, targets, img_paths, indices=None):
    counter_inst = np.ones((num_categories), np.int)
    if indices == None:
        indices = xrange(len(targets))
        
    for j in indices:
        category_ind = targets[j]
        img_dir = os.path.join(split_dir, str(category_ind))
        
        num_img = counter_inst[category_ind]
        dst_basename = str(num_img).zfill(padding_zeros)
        dst_basename += ext(img_paths[j])
        dst_filepath = os.path.join(img_dir, dst_basename)

        os.symlink(img_paths[j], dst_filepath)
        counter_inst[category_ind] += 1

def read_split(filepath):
    days = list()
    with open(filepath) as f:
        for line in f.readlines():
            user_id, date = line.replace("\n", "").split(' ')
            days.append((user_id, date))
    return days

In [26]:
num_categories = len(categories)
num_images = sum([u.num_images for u in users])
padding_zeros = num_digits(num_images)

splits = ['training','test','validation']
filepaths = ['training_split.txt','validation_split.txt', 'test_split.txt']
for split, filepath in zip(splits, filepaths):
    
    split_dir = osp.join('data', split)
    if os.path.isdir(split_dir):
        shutil.rmtree(split_dir)    
    
    for j in xrange(num_categories): 
        category = str(j)
        category_dir = os.path.join(split_dir, category)
        makedirs(category_dir)
        
    targets = list()
    img_paths = list()
    for user_id, date in read_split(filepath):
        for image in days[user_id][date].images:
            targets.append(image.label)
            img_paths.append(image.path)
    
    link_images(num_categories, split_dir, padding_zeros, targets, img_paths)